#### This is a naive implementation of pagerank algorithm with no optimizations. The matrix operations happen in a dense manner and as we keep increasing the size of the graph, it becomes computationally intensive to compute the transition matrix and followed by the pagerank of each node.

#### This is for purely getting the concept of the algorithm right and implementing it without any flaws.

***Note : if the graph is too large, you can always store the adjacency matrix of it on disk and directly load parts of it from there using memmap functionality.***

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

def create_transition_matrix(adjacency_matrix, num_nodes, damping_factor=0.85):
    adjacency_matrix = adjacency_matrix.copy()
    row_sums = adjacency_matrix.sum(axis=1)

    for i in range(num_nodes):
        if row_sums[i] == 0:
            #if a node has no outgoing edges, assume it links to all other nodes
            adjacency_matrix[i] = 1 / num_nodes
        else:
            #normalize rows for nodes with outgoing edges
            adjacency_matrix[i] /= row_sums[i]

    #apply damping factor to the transition matrix
    transition_matrix = damping_factor * adjacency_matrix + ((1 - damping_factor) / num_nodes) * np.ones((num_nodes, num_nodes), dtype = np.float32)

    return transition_matrix


# ***Here, I am to creating the transition matrix in a batchwise fashion:***
so that if the adjacency matrix is too large, only a few of the rows will be fetched and
they will be : <br>
a) Normalized <br>
b) Modified with the damping factor.

**Caution : The transition matrix created will modify the adjacency matrix(which is passed as the parameter) in-place. And it is advised to store the adjacency matrix on disk instead of main memory with the help of the memmap functionality.**


In [2]:
def batch_indices(start, stop, batch_size):
    current = start
    while current <= stop:
        end = min(current + batch_size-1, stop)
        yield (current, end)
        current += batch_size

def create_transition_matrix_batchwise(adjacency_matrix, num_nodes, damping_factor=0.85, batch_size = 400):
    for start_idx, end_idx in tqdm(batch_indices(0, num_nodes-1, batch_size), desc = "computing transition matrix", total = np.ceil(num_nodes/batch_size)):

        row_sums = adjacency_matrix[start_idx:end_idx + 1].sum(axis = 1)

        for i in range(start_idx, end_idx + 1):
            if row_sums[i - start_idx] == 0:
                # If a node has no outgoing edges, assume it links to all other nodes
                adjacency_matrix[i] = 1 / num_nodes
            else:
                # Normalize rows for nodes with outgoing edges
                adjacency_matrix[i] /= row_sums[i - start_idx]

        t_mtx = damping_factor * adjacency_matrix[start_idx:end_idx+1] + ((1 - damping_factor) / num_nodes) * np.ones(((end_idx - start_idx + 1), num_nodes))
        np.copyto(adjacency_matrix[start_idx:end_idx+1], t_mtx, casting = 'same_kind')

    return adjacency_matrix

def compute_pagerank(transition_matrix, num_nodes, tol=1e-6, max_iter=10000):
    # Initialize the PageRank vector with equal probability for each node
    pagerank = np.ones(num_nodes) / num_nodes

    for _ in range(max_iter):
        new_pagerank = transition_matrix.T @ pagerank  # Transpose the transition matrix here
        # Check for convergence
        if np.linalg.norm(new_pagerank - pagerank, 1) < tol:
            break
        pagerank = new_pagerank

    return pagerank

# Main Code

In [4]:
nodes_file = '/content/sample_nodes_0.005_204_pageranked_gephi.csv'
edges_file = '/content/sample_edges_0.005_204.csv'

nodes_df = pd.read_csv(nodes_file)
edges_df = pd.read_csv(edges_file)

node_ids = nodes_df['Id'].values
id_to_index = {node_id: idx for idx, node_id in enumerate(node_ids)}
num_nodes = len(node_ids)


to_disk = True
adj_mtx_path = './adjacency_matrix.dat'
adjacency_matrix = None

if(to_disk):
    adjacency_matrix = np.memmap(adj_mtx_path, dtype = 'float32', mode = 'w+', shape = (num_nodes, num_nodes))
    for _, row in edges_df.iterrows():
        src_idx = id_to_index[row['Source']]
        tgt_idx = id_to_index[row['Target']]
        adjacency_matrix[src_idx, tgt_idx] = 1.0

    #flush any remaining write operations to disk and delete the reference
    adjacency_matrix.flush()
    del adjacency_matrix

else:
    adjacency_matrix = np.zeros((num_nodes, num_nodes), dtype = np.float32)
    for _, row in edges_df.iterrows():
        src_idx = id_to_index[row['Source']]
        tgt_idx = id_to_index[row['Target']]
        adjacency_matrix[src_idx, tgt_idx] = 1.0

In [5]:
if(to_disk):
    adjacency_matrix = np.memmap(adj_mtx_path, dtype = 'float32', mode = 'r+', shape = (num_nodes, num_nodes))

# Create the transition matrix with damping factor
damping_factor = 0.85
transition_matrix = None
if(to_disk):
    batch_size = 174
    transition_matrix = create_transition_matrix_batchwise(adjacency_matrix, num_nodes, damping_factor, batch_size)
    #flush any remaining disk write operations from creating the transition matrix
    transition_matrix.flush()
else:
    transition_matrix = create_transition_matrix(adjacency_matrix, num_nodes, damping_factor)


# Compute PageRank
pagerank = compute_pagerank(transition_matrix, num_nodes)
print("PageRank values:", pagerank)

computing transition matrix:   0%|          | 0/52.0 [00:00<?, ?it/s]

PageRank values: [1.87012065e-05 9.86937234e-04 9.84473621e-03 ... 1.83324213e-05
 1.83324213e-05 1.83324213e-05]


In [6]:
node_ids = nodes_df['Id'].values
nodes_df['my_pagerank'] = nodes_df['Id'].map(lambda x : pagerank[id_to_index[x]])

In [9]:
nodes_df.sample(30)

,Id,Label,names,pageranks,my_pagerank
6155,1108993,NaN,Armenians in Italy,0.000018,0.000018
4495,713177,NaN,Faith No More discography,0.000018,0.000018
1129,84320,NaN,Southern Football League,0.000153,0.000153
5989,1190374,NaN,Baraka (film),0.000040,0.000040
8440,202884,NaN,List of British Columbia Provincial Parks,0.000018,0.000018
8131,1726823,NaN,Project Athena,0.000018,0.000018
5527,1024151,NaN,San Francisco (Bobby Hutcherson album),0.000018,0.000018
264,1089360,NaN,Materials science,0.000342,0.000342
7733,1543537,NaN,Colorado Buffaloes football,0.000018,0.000018
2878,341917,NaN,Shamarpa,0.000018,0.000018


In [8]:
nodes_df[['pageranks', 'my_pagerank']].describe()

,pageranks,my_pagerank
count,8957.000000,8957.000000
mean,0.000111,0.000112
std,0.000413,0.000413
min,0.000018,0.000018
25%,0.000018,0.000018
50%,0.000018,0.000018
75%,0.000045,0.000045
max,0.016444,0.016445
